<a href="https://colab.research.google.com/github/s2young/ng-file-upload/blob/master/WindowOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install poppler-utils

import torch
use_gpu = torch.cuda.is_available()

!pip3 -q install --upgrade pip
if use_gpu:
 !python -m pip -q install paddlepaddle-gpu -i https://pypi.tuna.tsinghua.edu.cn/simple
else:
  !python -m pip -q install paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple

!pip -q install -U sahi
!pip -q install pdf2image
!pip -q install ultralytics
!pip -q install "paddleocr>=2.0.1"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (329 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 123629 

In [ ]:
!gdown 1-jxQaIQaexQHz-BwxgCeCaElu2V539i8

In [ ]:
import os
import cv2
import json
import numpy as np
from glob import glob
from ultralytics import YOLO
from paddleocr import PaddleOCR
from pdf2image import convert_from_path

from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.utils.yolov8 import download_yolov8m_model
from sahi.predict import get_sliced_prediction, predict


ocr = PaddleOCR(use_angle_cls=True, lang='en') # or ch

# Download YOLOv8 model
yolov8_model_path = "bestWindow_v8m.pt"


detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=yolov8_model_path,
    confidence_threshold=0.3,
    device='cuda:0',  # or 'cuda:0',
    image_size=608,
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:15<00:00, 248.71it/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10000/10000 [00:17<00:00, 586.22it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:14<00:00, 150.02it/s]

[2024/11/15 09:50:04] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_ch

[2024/11/15 09:50:05] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2024/11/15 09:50:06] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0


In [ ]:
def detect_ocr(file_path, detection_model, save_dir='detect_ocr'):

    os.makedirs(f'{save_dir}', exist_ok=True)

    images = convert_from_path(file_path)

    for page_idx, image in enumerate(images):

        img = np.array(image)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        dets = []

        results = get_sliced_prediction(
                    img,
                    detection_model,
                    slice_height=600,
                    slice_width=600,
                    overlap_height_ratio=0.25,
                    overlap_width_ratio=0.25
                    ).object_prediction_list

        if results != []:
            for result in results:
                x1, y1, x2, y2 = result.bbox.to_xyxy()
                x = int((x1 + x2)/2)
                y = int((y1 + y2)/2)
                w = int(x2 - x1)
                h = int(y2 - y1)

                if h < w:
                    new_x1 = x - w
                    new_x2 = x + w
                    new_y1 = y - 2*h
                    new_y2 = y + 2*h
                    cropped = img[new_y1:new_y2, new_x1:new_x2]

                else:
                    new_x1 = x - 2*w
                    new_x2 = x + 2*w
                    new_y1 = y - h
                    new_y2 = y + h
                    cropped = img[new_y1:new_y2, new_x1:new_x2]
                    cropped = cv2.rotate(cropped, cv2.ROTATE_90_CLOCKWISE)

                if cropped is not None:
                    txts = []
                    if cropped.shape[0] * cropped.shape[1] != 0:
                        result1 = ocr.ocr(cropped, cls=True)[0]
                        if result1 is not None:
                            locs1 = [line[0] for line in result1]
                            txts1 = [line[1][0] for line in result1]
                            confs1 = [line[1][1] for line in result1]
                            for idx in range(len(locs1)):
                                txt = txts1[idx]
                                conf = confs1[idx]
                                xs, ys = zip(*locs1[idx])
                                x11 = int(min(xs))
                                y11 = int(min(ys))
                                x22 = int(max(xs))
                                y22 = int(max(ys))

                                if h >= w:
                                    h1, w1 = cropped.shape[:2]
                                    new_x11 = y11
                                    new_y11 = w1 - x22
                                    new_x22 = y22
                                    new_y22 = w1 - x11

                                    new_x11 += new_x1
                                    new_y11 += new_y1
                                    new_x22 += new_x1
                                    new_y22 += new_y1

                                else:
                                    new_x11 = x11 + new_x1
                                    new_y11 = y11 + new_y1
                                    new_x22 = x22 + new_x1
                                    new_y22 = y22 + new_y1

                                cv2.rectangle(img, (new_x11, new_y11), (new_x22, new_y22), (255, 0, 0), 1)
                                cv2.putText(img, txt, (new_x11, new_y11-3), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.7, (255, 0, 0), 1)
                                txts.append(txt)

                    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 255), 2)

                    for txt in txts:
                        txt = txt.replace('|', '').replace(' ', '')

                        if len(txt) in [9, 10] and txt[0] == '(' and txt[1].isdigit():
                            try:
                                num = int(txt[1])
                                width = int(txt[3:5])
                                height = int(txt[5:7])
                                style = txt[7:]

                                dets.append({'loc':[x1, y1, x2, y2],
                                            'num':num,
                                            'width':width,
                                            'height':height,
                                            'style':style})

                                break

                            except:
                                pass



                        elif len(txt) in [6, 7] and txt[0].isdigit():
                            try:
                                width = int(txt[:2])
                                height = int(txt[2:4])
                                style = txt[4:]

                                dets.append({'loc':[x1, y1, x2, y2],
                                            'num':1,
                                            'width':width,
                                            'height':height,
                                            'style':style})

                            except:
                                pass



        with open(f'{save_dir}/{page_idx+1}_pred.json', 'w') as f:
            json.dump(dets, f)

        cv2.imwrite(f'{save_dir}/{page_idx+1}_pred.png', img)

In [ ]:
file_path = ''
detect_ocr(file_path, detection_model, save_dir='detect_ocr')